In [3]:
from datasets import load_dataset
food = load_dataset("food101", split="train[:5000]")

In [10]:
pip install ipywidgets

                                              0.0/139.4 kB ? eta -:--:--
     --------                              30.7/139.4 kB 660.6 kB/s eta 0:00:01
     ----------------                      61.4/139.4 kB 656.4 kB/s eta 0:00:01
     -----------------------------        112.6/139.4 kB 939.4 kB/s eta 0:00:01
     ------------------------------------ 139.4/139.4 kB 916.1 kB/s eta 0:00:00
                                              0.0/2.3 MB ? eta -:--:--
     -                                        0.1/2.3 MB 3.3 MB/s eta 0:00:01
     --                                       0.2/2.3 MB 1.8 MB/s eta 0:00:02
     ----                                     0.2/2.3 MB 2.1 MB/s eta 0:00:01
     ------                                   0.4/2.3 MB 1.9 MB/s eta 0:00:02
     ---------                                0.5/2.3 MB 2.2 MB/s eta 0:00:01
     ---------                                0.6/2.3 MB 2.2 MB/s eta 0:00:01
     ---------                                0.6/2.3 MB 2.2 MB/s et


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import huggingface_hub

huggingface_hub.login()
# huggingface_hub.notebook_login()

In [8]:
food = food.train_test_split(test_size=0.2)

In [9]:
food["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 'label': 10}

In [10]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
    

In [11]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [12]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [13]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples
food = food.with_transform(transforms)

평가

In [14]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [15]:
import evaluate

accuracy = evaluate.load("accuracy")

In [16]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

훈련

In [17]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir="my_awesome_food_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/186 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [19]:
trainer.push_to_hub()




training_args.bin: 100%|██████████| 4.73k/4.73k [00:00<00:00, 13.4kB/s]
model.safetensors: 100%|██████████| 344M/344M [00:38<00:00, 8.83MB/s]

Upload 2 LFS files: 100%|██████████| 2/2 [00:39<00:00, 19.67s/it]


CommitInfo(commit_url='https://huggingface.co/hiddenbebb/my_awesome_food_model/commit/3182b6bb3d4482800c18704756a384a84cce94c7', commit_message='End of training', commit_description='', oid='3182b6bb3d4482800c18704756a384a84cce94c7', pr_url=None, pr_revision=None, pr_num=None)

추론

In [ ]:
ds = load_dataset("food101", split="validation[:10]")
image = ds["image"][0]

In [1]:
from transformers import pipeline

classifier = pipeline("image-classification", model="my_awesome_food_model")
classifier(image)

print(image)

c:\Users\OWNER\Desktop\coding_project\ai_study2\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'image' is not defined

In [22]:
from transformers import AutoImageProcessor
import torch

image_processor = AutoImageProcessor.from_pretrained("my_awesome_food_model")
inputs = image_processor(image, return_tensors="pt")

In [23]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("my_awesome_food_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [24]:
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]

'falafel'